In [11]:
import sys
sys.path.insert(1, ".")

from Utils import Utils
import numpy as np
from sklearn.feature_extraction import DictVectorizer

In [2]:
vocab = Utils.file_to_vocab("./data/en-train.txt")

In [3]:
# simple baseline
#Utils.gen_baseline_labels("./data/en-test.txt", "./simple_baseline_test.txt", vocab)

In [4]:
train_Y = Utils.extract_labels('./data/en-train.txt')
train_Y = np.array(train_Y)

In [5]:
# generate cosine feature on training data and save
# import pickle 
# with open('./data/en-train.txt','r') as f1:
#     lines = f1.readlines()

# # extract cosine feature
# cosine_sim_feature = Utils.gen_cosine_sim_feature(lines, vocab, stem=True)
# train_X = np.array(cosine_sim_feature)
# pickle.dump(train_X,open("feature_train_cosine.pickle","wb"))

In [6]:
# from Utils import Utils
# from sklearn.feature_extraction import DictVectorizer

# # generate NER and POS features
# features_by_row = Utils.gen_pos_ner_features('./data/en-train.txt')
# vectorizer = DictVectorizer()
# X_train_ner_pos = vectorizer.fit_transform(features_by_row)


In [7]:
# import pickle
# pickle.dump(X_train_ner_pos,open("feature_train_pos_ner.pickle","wb"))

In [8]:
#combine features if necessary into X_train

import pickle
import codecs
import numpy as np
from scipy import sparse

with codecs.open("feature_train_cosine.pickle",'rb', errors='ignore') as f1:
    cosine_feature = pickle.load(f1)
    cosine_feature = np.array([[i] for i in cosine_feature])
    
with codecs.open("feature_train_pos_ner.pickle",'rb', errors='ignore') as f1:
    pos_features = pickle.load(f1)
  
train_X = sparse.hstack([pos_features, cosine_feature])


In [44]:
# train KernelRidge model
# using 'krm' as all-purpose model name for simplicity
    
from sklearn.kernel_ridge import KernelRidge
krm = KernelRidge(alpha=0.5, gamma=.4, kernel='rbf')
krm.fit(train_X, train_Y) 


KernelRidge(alpha=0.5, coef0=1, degree=3, gamma=0.4, kernel='rbf',
      kernel_params=None)

In [40]:
# train RF regressor
# from sklearn.ensemble import RandomForestRegressor
# krm = RandomForestRegressor(max_depth=None, random_state=0, n_estimators=320)
# krm.fit(train_X, train_Y) 
# print(krm.feature_importances_)


In [45]:
mode = 'val'

val_Y = Utils.extract_labels('./data/en-' + mode + '.txt')
val_Y = np.array(val_Y)

with open('./data/en-' + mode + '.txt','r') as f1:
    lines = f1.readlines()

# extract cosine feature
cosine_sim_feature = Utils.gen_cosine_sim_feature(lines, vocab, stem=True)
cosine_sim_feature = np.array([[i] for i in cosine_sim_feature])
#val_X = np.array(cosine_sim_feature).reshape(-1,1)

features_by_row = Utils.gen_pos_ner_features('./data/en-' + mode + '.txt')
vectorizer = DictVectorizer()
X_val_ner_pos = vectorizer.fit_transform(features_by_row)
val_X = sparse.hstack([X_val_ner_pos, cosine_sim_feature])

val_Y_pred = krm.predict(val_X)

with open(mode + '_y_ner_pos_kr_rbf_pred.txt', 'w') as f:      
    for score in val_Y_pred:
        f.write("{}\n".format(score)) 

0 3.5355339059327373
1000 1.5430334996209194
0
1000
